In [ ]:
import os
from langgraph.graph import MessagesState, StateGraph, END, START
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import tools_condition # this is the checker for if you got a tool back
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from typing import Annotated, TypedDict
import operator
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
import yfinance as yf


In [ ]:
llm = ChatGroq(model_name="Gemma2-9b-It")

In [ ]:
def multiply(a: int, b:int) -> int:
    """
    Multiply a and b.
    Args:
        a: first int
        b: second int
    """

    return a * b

In [ ]:
def adding(a: int, b:int) -> int:
    """
    add a and b.
    Args:
        a: first int
        b: second int
    """

    return a + b

In [ ]:
def divide(a: int, b:int) -> float:
    """
    divide a and b.
    Args:
        a: first int
        b: second int
    """

    return a / b

In [ ]:
search = DuckDuckGoSearchRun()

In [ ]:
search.invoke("who is the president of USA?")

In [ ]:
tools_1 = [adding, multiply, divide, search]

In [ ]:
llm_with_tools = llm.bind_tools(tools_1)

In [ ]:
system_prompt = SystemMessage(content="You are a helpful assistant tasked with search and performing arithmetic operation on a set of inputs")

In [ ]:
def reasoner(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([system_prompt] + state["messages"])]}

In [ ]:
# Graph
workflow = StateGraph(MessagesState)

# Add nodes
workflow.add_node("reasoner", reasoner)
workflow.add_node("tools", ToolNode(tools_1)) # for the tools

# Add edge
workflow.add_edge(START, "reasoner")

workflow.add_conditional_edges(
    "reasoner",
    # if the latest message (result) from node reasoner is a tool call -> tools_condition routes to tools
    # if the latest message (result) from node reasoner is a not tool call -> tools_condition routes to END
    tools_condition,
)

workflow.add_edge("tools", "reasoner")

react_graph = workflow.compile()

In [ ]:
# Displaying the graph
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
messages = [HumanMessage(content="What is 2 times of average human age?")]

In [ ]:
messages = react_graph.invoke({"messages": messages})

In [ ]:
for m in messages["messages"]:
    m.pretty_print()

In [ ]:
def get_stock_price(ticker: str) -> float:
    """
    Gets stock price from Yahoo Finance.

    Args:
        ticker: ticker str
    """
    # This is a tool for getting the price of stock when passed a ticker symbol
    stock = yf.Ticker(ticker)
    return stock.info["previousClose"]

In [ ]:
get_stock_price("AAPL")

In [ ]:
# Node
def reasoner_2(state):
    query = state["query"]
    messages = state["messages"]

    # System message
    system_message = SystemMessage(content="You are a helpful assistant tasked with using search, the yahoo finance tool and perform arithmetic operations")
    message = HumanMessage(content=query)
    messages.append(messages)
    result = [llm_with_tools.invoke([system_message] + messages)]
    return {"messages": result}

In [ ]:
tools = [adding, multiply, divide, search, get_stock_price]

In [ ]:
llm_with_tools_1 = llm.bind_tools(tools)

In [ ]:
class GraphState(TypedDict):
    """ State of the graph """
    query = str
    finance = str
    final_answer = str
    # Intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
# Graph
workflow_1 = StateGraph(GraphState)

# Add nodes
workflow_1.add_node("reasoner_2", reasoner_2)
workflow_1.add_node("tools", ToolNode(tools=tools)) # For the tools

# Add Edges
workflow_1.add_edge(START, "reasoner_2")

workflow_1.add_conditional_edges(
    "reasoner_2",
    # if the latest message (result) from node reasoner is a tool call -> tools_condition routes to tools
    # if the latest message (result) from node reasoner is a not tool call -> tools_condition routes to END
    tools_condition
)

workflow_1.add_edge("tools", "reasoner_2")
react_graph_1 = workflow_1.compile()

In [ ]:
# Showing 
display(Image(react_graph_1.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
response = react_graph_1.invoke({"query": "What is the stock price of Apple adding with 1000?", "messages":[]})

In [ ]:
for m in response["messages"]:
    m.pretty_print()